<a href="https://colab.research.google.com/github/Raiden-Makoto/swiftualizer/blob/main/TransformerModel/SwiftNETV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SwiftNET:
**Keras + Tensorflow implementation of WaveNET specifically trained to generate pop songs in the style of Taylor Swift**

In [12]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.data as tf_data
import keras
from keras import layers
from keras import ops
from keras.initializers import GlorotUniform, Zeros

### Helper Operation for the main **SwiftNET** Model
Code is sourced from [here](https://github.com/kokeshing/WaveNet-tf2/blob/master/model/module.py) but modified to use `Keras 3`.

A causal convolution layer is a type of 1D convolution designed for sequential data where the output at time step $t$ depends only on inputs from time steps
$ \leq t$. This structure ensures that there is no "leakage" of future information into the past, making it useful for autoregressive models like WaveNet and time-series forecasting.

In [13]:
class CausalConvolutionLayer(layers.Conv1D):
    def __init__(
            self,
            filters,
            kernel_size,
            strides=1,
            padding='causal',
            dilation_rate=1,
            residual_channels=None,
            *args,
            **kwargs
        ):
        super().__init__(
            filters,
            kernel_size,
            strides=strides,
            padding=padding,
            dilation_rate=dilation_rate
        )
        self.k = kernel_size
        self.d = dilation_rate
        if kernel_size > 1:
            self.queue_len = kernel_size + (kernel_size - 1) * (dilation_rate - 1)
            self.queue_dim = residual_channels
            self.init_queue()

    def build(self, input_shape):
        super().build(input_shape)
        self.linearized_weights = ops.cast(
            ops.reshape(self.kernel, [-1, self.filters]),
            dtype=tf.float32
        )

    def call(self, inputs, is_synthesis=False):
        if not is_synthesis: return super().call(inputs)
        if self.k > 1:
            self.queue = self.queue[:, 1:, :]
            self.queue = ops.concatenate(
                [self.queue, ops.expand_dims(inputs[:, -1, :], axis=1)],
                axis=1
            )
            if self.d > 1: inputs = self.queue[:, 0::self.d, :]
            else: inputs = self.queue
        outputs = ops.matmul(ops.reshape(inputs, [1, -1]), self.linearized_weights)
        outputs = keras.backend.bias_add(outputs, self.bias)
        return tf.reshape(outputs, [-1, 1, self.filters])

    def init_queue(self):
        self.queue = ops.zeros([1, self.queue_len, self.queue_dim], dtype=tf.float32)

## Create the SwiftNET Model